In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import datetime
import csv
import time
import os
from operator import itemgetter
import pandas as pd
import xlsxwriter
import random

In [2]:
"""
1. you must have a stock list xlsx file in the folder with stocks that you want to pull data
2. you must have a folder called Stocks
"""

'\n1. you must have a stock list xlsx file in the folder with stocks that you want to pull data\n2. you must have a folder called Stocks\n'

In [3]:
today = str (date.today())
print(today)
# reading stock list
list_loc = "Stocks.xlsx"
#number of time that you want to repeat
n_repeat = 90
#location of the reports
path_3 = r"Reports\%sReport.xlsx" % today
# need to fix when more than one date is missing


2019-06-04


In [4]:
def timer(counter):
    if counter % 30 == 0 or counter % 25 == 0:
        print('waiting')
        time.sleep(0.002)
        
def ran_timer():
    time.sleep(random.random()/random.randrange(10,100))

def df_add(df, seq, i, last_date, last_closed):
    #adding data into column
    new_df = [seq, i, last_date, last_closed] 
    print(new_df)
    df.loc[-1] = new_df
    #fixing index
    df.index = df.index + 1 

def df_sort(df):
    df = df.sort_values(by = ['SEQ'], ascending = False, axis = 0)
    return df

    
def mine_data(i, finder):
    url = "https://finance.yahoo.com/quote/"+i+"/history"
    headers = {'User-Agent': 'Mozilla/5.0'}
    page = requests.get(url,headers=headers)
    bsobj = BeautifulSoup(page.content, "html.parser")
    data = bsobj.find("span",{"data-reactid":"%s" %finder})
    if data == None:
        data = None
    else:
        data = ' '.join([i.strip() for i in data])
        data = data.replace(",", "")
    return data

def int_date(date):
    date = str(date.replace(',',''))
    return int(datetime.datetime.strptime(date, "%b %d %Y").strftime('%Y%m%d'))

In [5]:
class finder:
    def __init__(self, loc):
        self.loc = loc
        self.name = self
        
    def add(self, x):
        self.loc += x
        
    #next day for normal stocks
    def n_s_n(self):
        x = 15
        self.add(x)
        
    #next day for normal funds
    def n_f_n(self):
        y = 14
        self.add(y)
    
    #next day for stocks with dividend 
    def d_s_n(self):
        z = 22
        self.add(z)
        
    #next day for funds with dividend 
    def d_f_n(self):
        k = 21
        self.add(k)
    
    def empty_date(self):
        x = -1
        self.add(x)

In [6]:
def new_data(n_repeat, i, path_2):
    #creating a new file and dataframe
    print('New Stock: %s'%i)
    df_n = pd.DataFrame(columns = ['SEQ', 'Stock Name', 'Date', 'Closed Price'])
    
    #setting
    last_date_finder = finder(51)
    last_closed_finder = finder(61)
    second_date_finder = finder(66)

    n_repeat = n_repeat
    counter = 0
    fund_finder = False
    
    #Data minning
    while n_repeat > 0:
        counter += 1
        if counter %30 == 0 or counter %25 ==0:
            print('waiting')
            timer(counter)
            print(counter)
        else:
            print(counter)   
        
        last_date = mine_data(i, last_date_finder.loc)
        seq = int_date(last_date)
        
        second_date = mine_data(i, second_date_finder.loc)
        if second_date == None:
            print('Second date empty')
            ran_timer()
            print('Funds')
            # moving it to next date
            if fund_finder == False:
                second_date_finder.empty_date()
            fund_finder = True
            print(second_date_finder.loc)
            second_date = mine_data(i, second_date_finder.loc)
        seq_s = int_date(second_date)  
        
        #normal setting
        if seq != seq_s:
            last_closed = mine_data(i, last_closed_finder.loc)
            df_add(df_n, seq, i, last_date, last_closed)
            ran_timer()
            n_repeat -= 1
            if fund_finder == True:
                last_date_finder.n_f_n()
                last_closed_finder.n_f_n()
                second_date_finder.n_f_n()
            else:
                last_date_finder.n_s_n()
                last_closed_finder.n_s_n()
                second_date_finder.n_s_n()
                
        elif seq == seq_s:
            print('Dividend')
            last_closed = mine_data(i, last_closed_finder.loc)
            df_add(df_n, seq, i, last_date, last_closed)
            n_repeat -= 1
            
            if fund_finder == True:
                last_date_finder.d_f_n()
                last_closed_finder.d_f_n()
                second_date_finder.d_f_n()
            else:
                last_date_finder.d_s_n()
                last_closed_finder.d_s_n()
                second_date_finder.d_s_n()
        else:
            print('Error')      
    df_n.to_csv(path_2) 

In [7]:
def old_data(n_repeat, i, path_2):
    #reading an exisiting file
    print('Updating existing stock: %s'%i)
    df = pd.read_csv(path_2)
    df = df[['SEQ', 'Stock Name', 'Date', 'Closed Price']]
    
    #setting
    last_date_finder = finder(51)
    last_closed_finder = finder(61)
    second_date_finder = finder(66)
    counter = 0
    fund_finder = False
    seq_counter = 0
    seq_o = df['SEQ'][seq_counter]
    
    last_date = mine_data(i, last_date_finder.loc)
    seq = int_date(last_date)
    second_date = mine_data(i, second_date_finder.loc)
    if second_date == None:
        seq_s = int_date(mine_data(i, second_date_finder.loc - 1))
    else:
        seq_s = int_date(second_date)
    
    if seq-seq_o > 10:
        print("data missing")
        os.remove(path_2)
        ran_timer()
        print('csv file deleted')
        new_data(n_repeat, i, path_2)
        
    else:
        
        while seq_o != seq:
            last_date = mine_data(i, last_date_finder.loc)
            seq = int_date(last_date)
            second_date = mine_data(i, second_date_finder.loc)

            #stocks with dividend
            if second_date == None:
                print('second date empty')
                ran_timer()
                second_date = mine_data(i, second_date_finder.loc)
                print('funds')
                second_date_finder.empty_date()
                fund_finder = True
                second_date = mine_data(i, second_date_finder.loc)

            first_closed_finder = finder(61)
            first_closed = mine_data(i, first_closed_finder.loc)

            if first_closed == None:
                print('Dividend for wmt')
                ran_timer()
                last_date_finder = finder(58)
                last_closed_finder = finder(66)
                second_date_finder = finder(73)
                second_date = mine_data(i, second_date_finder.loc)
                last_closed = mine_data(i, last_date_finder.loc)

            seq_s = int_date(second_date)

            #normal stocks
            if seq != seq_s:
                last_closed = mine_data(i, last_closed_finder.loc)
                if first_closed != None:
                    
                    df_add(df, seq, i, last_date, last_closed)
                    df = df_sort(df)
                    
                    seq_counter += 1
                    seq_o = df['SEQ'][seq_counter]
                    
                    ran_timer()
                    
                    if fund_finder == True:
                        last_date_finder.n_f_n()
                        last_closed_finder.n_f_n()
                        second_date_finder.n_f_n()
                    else:
                        last_date_finder.n_s_n()
                        last_closed_finder.n_s_n()
                        second_date_finder.n_s_n()

            elif seq == seq_s:
                print(seq)
                print(seq_s)
                print('Dividend')
                last_closed = mine_data(i, last_closed_finder.loc)
                
                df_add(df, seq, i, last_date, last_closed)
                df_sort(df)
                
                seq_counter += 1
                seq_o = df['SEQ'][seq_counter]
                
                last_date_finder.d_s_n()
                last_closed_finder.d_s_n()
                second_date_finder.d_s_n()
        df = df.drop_duplicates(subset = ['SEQ'], keep = "last")
        df.to_csv(path_2)

In [8]:
def analyze_data(df_f, stock_list, columns):
    for i in stock_list:
        path_2 = "Stocks\%s.csv" %i
        df_o = pd.read_csv(path_2)
        df_f = df_f
        current_seq = df_o.loc[0,"SEQ"]
        current_price = df_o.loc[0,"Closed Price"]
        # creating averages
        days = list(range(10,91,10))
        aver ={}
        for day in days:
            aver[day] = 1-(df_o.loc[0:(day-1),"Closed Price"].mean()/current_price)
        df_f.loc[len(df_f)]=[current_seq,stock_dic_type[i],i,stock_dic[i], 
                             note_dic[i], current_price, buy_dic[i], 
                             (current_price - buy_dic[i])/ buy_dic[i],
                             aver[10],aver[20],aver[30],aver[40],aver[50],aver[60],aver[70],aver[80],aver[90]]
    return df_f

In [9]:
def organize_report(df_f, path_3, stock_list):
    #formating data
    df_f = df_f.sort_values(by=['Stock_Type', '10A'])
    print(df_f)
    path_3 = path_3
    writer = pd.ExcelWriter(path_3, engine='xlsxwriter')
    df_f.to_excel(writer, sheet_name='Sheet1')
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    format_data = workbook.add_format({'num_format': '0.00%'})
    worksheet.set_column('I2:Z99', None, format_data)

    format1 = workbook.add_format({'bg_color':   '#FFC7CE','font_color': '#9C0006'})
    worksheet.conditional_format('I2:Z99', {'type': 'cell', 'criteria': '<', 'value': 0, 'format': format1})

    worksheet.set_column('E:E', 23)

    format2 = workbook.add_format({'bg_color':   '#FFFF00'})
    worksheet.conditional_format('C1:C999', {'type': 'text', 'criteria': 'containing', 'value' : 'Index', 'format': format2})

    format3 = workbook.add_format({'bg_color':   '#008000'})
    worksheet.conditional_format('C1:C999', {'type': 'text', 'criteria': 'containing', 'value' : 'Owned', 'format': format3})

    writer.save()

    #open a file

    os.startfile(path_3)

In [10]:
#setting
df1 = pd.read_excel(list_loc)
stock_list = df1["Stocks"]
stock_names = df1["Stock_Name"]
stock_dic = dict(zip(stock_list, stock_names))
stock_type = df1["Stock_Type"]
stock_dic_type = dict(zip(stock_list,stock_type))
buy_price = df1["Buy_Price"]
buy_dic = dict(zip(stock_list, buy_price))
note = df1["Note"]
note_dic = dict(zip(stock_list, note))

In [11]:
# excuting functions
for i in stock_list:
    # if it is a new stock
    #location of stock files
    path_2 = "Stocks\%s.csv" %i
    if os.path.isfile(path_2) == False:
        new_data(n_repeat, i, path_2)
    else:
        old_data(n_repeat, i, path_2)

print('Analyzing all stock data')
columns = ['Date', 'Stock_Type', 'Stock_Symbol', 'Stock_Name', 'Note', 'Current Price', 'Buy Price', 'Gain / Loss',  '10A', '20A', '30A','40A','50A','60A','70A','80A','90A']        
df_f = pd.DataFrame(columns = columns)
df_f = analyze_data(df_f, stock_list, columns)
    
organize_report(df_f, path_3, stock_list)


Updating existing stock: ^DJI
Updating existing stock: ^IXIC
Updating existing stock: ^GSPC
Updating existing stock: GE
Updating existing stock: BEARX
Updating existing stock: FSAGX
Updating existing stock: FTQGX
New Stock: WMT
1
[20190603, 'WMT', 'Jun 03 2019', '101.96']
2
[20190531, 'WMT', 'May 31 2019', '101.44']
3
[20190530, 'WMT', 'May 30 2019', '102.19']
4
[20190529, 'WMT', 'May 29 2019', '102.12']
5
[20190528, 'WMT', 'May 28 2019', '102.42']
6
[20190524, 'WMT', 'May 24 2019', '102.67']
7
[20190523, 'WMT', 'May 23 2019', '101.86']
8
[20190522, 'WMT', 'May 22 2019', '102.23']
9
[20190521, 'WMT', 'May 21 2019', '101.12']
10
[20190520, 'WMT', 'May 20 2019', '101.52']
11
[20190517, 'WMT', 'May 17 2019', '100.86']
12
[20190516, 'WMT', 'May 16 2019', '101.31']
13
[20190515, 'WMT', 'May 15 2019', '99.88']
14
[20190514, 'WMT', 'May 14 2019', '100.29']
15
[20190513, 'WMT', 'May 13 2019', '99.89']
16
[20190510, 'WMT', 'May 10 2019', '101.91']
17
Dividend
[20190509, 'WMT', 'May 09 2019', '9

[20190211, 'FHKCX', 'Feb 11 2019', '32.63']
79
[20190208, 'FHKCX', 'Feb 08 2019', '32.32']
80
[20190207, 'FHKCX', 'Feb 07 2019', '32.22']
81
[20190206, 'FHKCX', 'Feb 06 2019', '32.68']
82
[20190205, 'FHKCX', 'Feb 05 2019', '32.89']
83
[20190204, 'FHKCX', 'Feb 04 2019', '32.49']
84
[20190201, 'FHKCX', 'Feb 01 2019', '32.38']
85
[20190131, 'FHKCX', 'Jan 31 2019', '32.48']
86
[20190130, 'FHKCX', 'Jan 30 2019', '32.15']
87
[20190129, 'FHKCX', 'Jan 29 2019', '31.57']
88
[20190128, 'FHKCX', 'Jan 28 2019', '31.73']
89
[20190125, 'FHKCX', 'Jan 25 2019', '31.91']
waiting
waiting
90
[20190124, 'FHKCX', 'Jan 24 2019', '31.19']
Updating existing stock: FSCHX
data missing
csv file deleted
New Stock: FSCHX
1
Second date empty
Funds
65
[20190603, 'FSCHX', 'Jun 03 2019', '12.12']
2
[20190531, 'FSCHX', 'May 31 2019', '11.60']
3
[20190530, 'FSCHX', 'May 30 2019', '11.83']
4
[20190529, 'FSCHX', 'May 29 2019', '11.90']
5
[20190528, 'FSCHX', 'May 28 2019', '11.91']
6
[20190524, 'FSCHX', 'May 24 2019', '11.

[20190301, 'FSCOX', 'Mar 01 2019', '17.83']
66
[20190228, 'FSCOX', 'Feb 28 2019', '17.74']
67
[20190227, 'FSCOX', 'Feb 27 2019', '17.77']
68
[20190226, 'FSCOX', 'Feb 26 2019', '17.85']
69
[20190225, 'FSCOX', 'Feb 25 2019', '17.80']
70
[20190222, 'FSCOX', 'Feb 22 2019', '17.69']
71
[20190221, 'FSCOX', 'Feb 21 2019', '17.65']
72
[20190220, 'FSCOX', 'Feb 20 2019', '17.67']
73
[20190219, 'FSCOX', 'Feb 19 2019', '17.63']
74
[20190215, 'FSCOX', 'Feb 15 2019', '17.54']
waiting
waiting
75
[20190214, 'FSCOX', 'Feb 14 2019', '17.35']
76
[20190213, 'FSCOX', 'Feb 13 2019', '17.30']
77
[20190212, 'FSCOX', 'Feb 12 2019', '17.24']
78
[20190211, 'FSCOX', 'Feb 11 2019', '17.04']
79
[20190208, 'FSCOX', 'Feb 08 2019', '17.00']
80
[20190207, 'FSCOX', 'Feb 07 2019', '17.15']
81
[20190206, 'FSCOX', 'Feb 06 2019', '17.30']
82
[20190205, 'FSCOX', 'Feb 05 2019', '17.37']
83
[20190204, 'FSCOX', 'Feb 04 2019', '17.19']
84
[20190201, 'FSCOX', 'Feb 01 2019', '17.06']
85
[20190131, 'FSCOX', 'Jan 31 2019', '17.06']


[20190320, 'LETRX', 'Mar 20 2019', '31.93']
53
[20190319, 'LETRX', 'Mar 19 2019', '31.60']
54
[20190318, 'LETRX', 'Mar 18 2019', '31.54']
55
[20190315, 'LETRX', 'Mar 15 2019', '31.09']
56
[20190314, 'LETRX', 'Mar 14 2019', '30.86']
57
[20190313, 'LETRX', 'Mar 13 2019', '30.95']
58
[20190312, 'LETRX', 'Mar 12 2019', '30.89']
59
[20190311, 'LETRX', 'Mar 11 2019', '30.82']
waiting
waiting
60
[20190308, 'LETRX', 'Mar 08 2019', '30.54']
61
[20190307, 'LETRX', 'Mar 07 2019', '30.67']
62
[20190306, 'LETRX', 'Mar 06 2019', '31.02']
63
[20190305, 'LETRX', 'Mar 05 2019', '30.94']
64
[20190304, 'LETRX', 'Mar 04 2019', '30.98']
65
[20190301, 'LETRX', 'Mar 01 2019', '30.86']
66
[20190228, 'LETRX', 'Feb 28 2019', '30.92']
67
[20190227, 'LETRX', 'Feb 27 2019', '30.94']
68
[20190226, 'LETRX', 'Feb 26 2019', '31.18']
69
[20190225, 'LETRX', 'Feb 25 2019', '31.25']
70
[20190222, 'LETRX', 'Feb 22 2019', '31.31']
71
[20190221, 'LETRX', 'Feb 21 2019', '31.00']
72
[20190220, 'LETRX', 'Feb 20 2019', '31.15']


[20190405, 'COMVX', 'Apr 05 2019', '4.3900']
41
[20190404, 'COMVX', 'Apr 04 2019', '4.4200']
42
[20190403, 'COMVX', 'Apr 03 2019', '4.4300']
43
[20190402, 'COMVX', 'Apr 02 2019', '4.4400']
44
[20190401, 'COMVX', 'Apr 01 2019', '4.4400']
45
[20190329, 'COMVX', 'Mar 29 2019', '4.4900']
46
[20190328, 'COMVX', 'Mar 28 2019', '4.5200']
47
[20190327, 'COMVX', 'Mar 27 2019', '4.5600']
48
[20190326, 'COMVX', 'Mar 26 2019', '4.5400']
49
[20190325, 'COMVX', 'Mar 25 2019', '4.5800']
waiting
waiting
50
[20190322, 'COMVX', 'Mar 22 2019', '4.5700']
51
[20190321, 'COMVX', 'Mar 21 2019', '4.4700']
52
[20190320, 'COMVX', 'Mar 20 2019', '4.5200']
53
[20190319, 'COMVX', 'Mar 19 2019', '4.5000']
54
[20190318, 'COMVX', 'Mar 18 2019', '4.4900']
55
[20190315, 'COMVX', 'Mar 15 2019', '4.5100']
56
[20190314, 'COMVX', 'Mar 14 2019', '4.5400']
57
[20190313, 'COMVX', 'Mar 13 2019', '4.5300']
58
[20190312, 'COMVX', 'Mar 12 2019', '4.5600']
59
[20190311, 'COMVX', 'Mar 11 2019', '4.5800']
waiting
waiting
60
[2019030

[20190425, 'MCD', 'Apr 25 2019', '196.78']
28
[20190424, 'MCD', 'Apr 24 2019', '196.48']
29
[20190423, 'MCD', 'Apr 23 2019', '194.12']
waiting
waiting
30
[20190422, 'MCD', 'Apr 22 2019', '192.79']
31
[20190418, 'MCD', 'Apr 18 2019', '193.78']
32
[20190417, 'MCD', 'Apr 17 2019', '190.49']
33
[20190416, 'MCD', 'Apr 16 2019', '190.58']
34
[20190415, 'MCD', 'Apr 15 2019', '190.73']
35
[20190412, 'MCD', 'Apr 12 2019', '190.48']
36
[20190411, 'MCD', 'Apr 11 2019', '187.78']
37
[20190410, 'MCD', 'Apr 10 2019', '188.12']
38
[20190409, 'MCD', 'Apr 09 2019', '188.97']
39
[20190408, 'MCD', 'Apr 08 2019', '188.74']
40
[20190405, 'MCD', 'Apr 05 2019', '189.60']
41
[20190404, 'MCD', 'Apr 04 2019', '188.76']
42
[20190403, 'MCD', 'Apr 03 2019', '187.25']
43
[20190402, 'MCD', 'Apr 02 2019', '187.25']
44
[20190401, 'MCD', 'Apr 01 2019', '187.29']
45
[20190329, 'MCD', 'Mar 29 2019', '188.79']
46
[20190328, 'MCD', 'Mar 28 2019', '188.29']
47
[20190327, 'MCD', 'Mar 27 2019', '186.42']
48
[20190326, 'MCD', 

[20190510, 'FSUTX', 'May 10 2019', '87.60']
17
[20190509, 'FSUTX', 'May 09 2019', '86.31']
18
[20190508, 'FSUTX', 'May 08 2019', '86.41']
19
[20190507, 'FSUTX', 'May 07 2019', '87.63']
20
[20190506, 'FSUTX', 'May 06 2019', '87.95']
21
[20190503, 'FSUTX', 'May 03 2019', '88.57']
22
[20190502, 'FSUTX', 'May 02 2019', '88.01']
23
[20190501, 'FSUTX', 'May 01 2019', '88.33']
24
[20190430, 'FSUTX', 'Apr 30 2019', '89.39']
waiting
waiting
25
[20190429, 'FSUTX', 'Apr 29 2019', '88.17']
26
[20190426, 'FSUTX', 'Apr 26 2019', '88.68']
27
[20190425, 'FSUTX', 'Apr 25 2019', '88.41']
28
[20190424, 'FSUTX', 'Apr 24 2019', '88.20']
29
[20190423, 'FSUTX', 'Apr 23 2019', '87.67']
waiting
waiting
30
[20190422, 'FSUTX', 'Apr 22 2019', '87.34']
31
[20190418, 'FSUTX', 'Apr 18 2019', '87.41']
32
[20190417, 'FSUTX', 'Apr 17 2019', '87.39']
33
[20190416, 'FSUTX', 'Apr 16 2019', '87.76']
34
[20190415, 'FSUTX', 'Apr 15 2019', '88.98']
35
[20190412, 'FSUTX', 'Apr 12 2019', '89.01']
36
[20190411, 'FSUTX', 'Apr 11 

[20190531, 'BRK-B', 'May 31 2019', '197.42']
3
[20190530, 'BRK-B', 'May 30 2019', '199.74']
4
[20190529, 'BRK-B', 'May 29 2019', '198.52']
5
[20190528, 'BRK-B', 'May 28 2019', '199.70']
6
[20190524, 'BRK-B', 'May 24 2019', '201.69']
7
[20190523, 'BRK-B', 'May 23 2019', '201.30']
8
[20190522, 'BRK-B', 'May 22 2019', '202.60']
9
[20190521, 'BRK-B', 'May 21 2019', '203.36']
10
[20190520, 'BRK-B', 'May 20 2019', '202.73']
11
[20190517, 'BRK-B', 'May 17 2019', '203.27']
12
[20190516, 'BRK-B', 'May 16 2019', '204.63']
13
[20190515, 'BRK-B', 'May 15 2019', '202.98']
14
[20190514, 'BRK-B', 'May 14 2019', '203.69']
15
[20190513, 'BRK-B', 'May 13 2019', '205.01']
16
[20190510, 'BRK-B', 'May 10 2019', '209.02']
17
[20190509, 'BRK-B', 'May 09 2019', '208.56']
18
[20190508, 'BRK-B', 'May 08 2019', '209.29']
19
[20190507, 'BRK-B', 'May 07 2019', '209.43']
20
[20190506, 'BRK-B', 'May 06 2019', '213.16']
21
[20190503, 'BRK-B', 'May 03 2019', '218.60']
22
[20190502, 'BRK-B', 'May 02 2019', '215.93']
23

[20190208, 'FDSVX', 'Feb 08 2019', '33.77']
80
[20190207, 'FDSVX', 'Feb 07 2019', '33.69']
81
[20190206, 'FDSVX', 'Feb 06 2019', '34.02']
82
[20190205, 'FDSVX', 'Feb 05 2019', '34.22']
83
[20190204, 'FDSVX', 'Feb 04 2019', '33.99']
84
[20190201, 'FDSVX', 'Feb 01 2019', '33.69']
85
[20190131, 'FDSVX', 'Jan 31 2019', '33.74']
86
[20190130, 'FDSVX', 'Jan 30 2019', '33.47']
87
[20190129, 'FDSVX', 'Jan 29 2019', '32.88']
88
[20190128, 'FDSVX', 'Jan 28 2019', '33.08']
89
[20190125, 'FDSVX', 'Jan 25 2019', '33.48']
waiting
waiting
90
[20190124, 'FDSVX', 'Jan 24 2019', '33.11']
Analyzing all stock data
        Date      Stock_Type Stock_Symbol  \
1   20190603           Index        ^IXIC   
2   20190603           Index        ^GSPC   
0   20190603           Index         ^DJI   
3   20190603           Index           GE   
19  20190603           Owned         AMZN   
6   20190603           Owned        FTQGX   
21  20190603           Owned        FDSVX   
13  20190603           Owned        FS